In [ ]:
#set path and mean file
#%%capture
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
import glob

caffe_root = '/home/Documents/Caffe/' 
import sys
sys.path.insert(0, caffe_root + 'python')

#os.environ['GLOG_minloglevel'] = '2'
import caffe

mean_filename='./mean.binaryproto'
proto_data = open(mean_filename, "rb").read()
a = caffe.io.caffe_pb2.BlobProto.FromString(proto_data)
mean  = caffe.io.blobproto_to_array(a)[0]

caffe.set_mode_gpu()

In [ ]:
#generate output from all 50 basic networks for a given set of images

labels = []
for ii in range(50):
    
    curIdx = ii+16
    age_net_pretrained='./basic_16to66/ot_'+str(curIdx)+'/basic_ot_'+str(curIdx)+'.caffemodel'
    age_net_model_file='./basic_16to66/deploy_basic.prototxt'
    age_net = caffe.Classifier(age_net_model_file, age_net_pretrained,
                           mean=mean,
                           channel_swap=(2,1,0),
                           raw_scale=255,
                           image_dims=(256, 256))

    labels.append([])
    for filename in sorted(glob.glob('./link_to_data/tr43k_te11k_16to66/album2_faces_crop_te11k/*.jpg')):
        
        im = caffe.io.load_image(filename)
        prediction = age_net.predict([im])
        label_tmp = prediction[0].argmax()
        labels[ind_lb].append(label_tmp)

In [ ]:
lb_all = np.array(labels) #binary labels
lb_sum = np.sum(lb_all, axis=0) #aggregated labels
lb_sum.shape

In [ ]:
#save results to txt files
np.savetxt('morph_te11k_results_bi.txt', lb_all, fmt='%d')

np.savetxt('morph_te11k_results_aggregate.txt', lb_sum, fmt='%d')

In [ ]:
#compare with ture labels and calculate MAE

load_in = np.genfromtxt('./link_to_data/tr43k_te11k_16to66/album2_faces_te11k_lb.txt', dtype=None)

lb_true = []
for i in range(len(load_in)):
    lb_true.append(load_in[i][1])

In [ ]:
lb_predict = lb_sum + 16

MAE_all = sum(abs(lb_true-lb_predict))*1.0/len(lb_true)
MAE_all